<a href="https://colab.research.google.com/github/camilanoeli/actividadGrupal1/blob/master/TP1/TP1_Parte1_Cpp_Python_Java_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **C++**

In [ ]:
%%writefile tp1.cpp
#include <iostream>
#include <unistd.h>
#include <vector>
#include <string>
#include <sys/wait.h>

// Estructura con la que armamos el árbol
typedef struct Node
{
    std::string name;
    std::vector<Node> children;
} Node;

int spawnProcesses(Node&);
void processMessage(Node& node)
{
    std::cout << "Hola mi nombre es " << node.name << ". PID: " << getpid() << " P_PID: "<< getppid() << std::endl;
}

int main()
{
    Node nodo = {"A", {{"B", {{"E"}, {"F"}}}, {"C"}, {"D", {{"G"}}}}};
    spawnProcesses(nodo);
    return EXIT_SUCCESS;
}


// Función recursiva encargada de crear los procesos hijos y esperarlos
int spawnProcesses(Node& node)
{
    int pid = 0;
    processMessage(node);
    for(auto& child : node.children)
    {
        pid = fork();
        if (pid == -1)
        {
            perror("fork");
            exit(EXIT_FAILURE);
        }
        else if(pid == 0)
        {
            spawnProcesses(child);
            sleep(50);
            return EXIT_SUCCESS;
        }
    }

    for(const auto& child : node.children)
    {
        wait(nullptr);
    }
    return EXIT_SUCCESS;
}




Overwriting tp1.cpp


In [ ]:
!touch resultado_cpp && chmod 700 resultado_cpp

In [ ]:
!g++ tp1.cpp -o tp1.out

In [ ]:
!nohup ./tp1.out 1>resultado_cpp 2>/dev/null &

In [ ]:
!cat resultado_cpp

Hola mi nombre es A. PID: 1856 P_PID: 1
Hola mi nombre es B. PID: 1857 P_PID: 1856
Hola mi nombre es C. PID: 1858 P_PID: 1856
Hola mi nombre es E. PID: 1860 P_PID: 1857
Hola mi nombre es D. PID: 1859 P_PID: 1856
Hola mi nombre es F. PID: 1861 P_PID: 1857
Hola mi nombre es G. PID: 1862 P_PID: 1859


In [ ]:
!ps -ef | grep tp1.out

root        1856       1  0 23:21 ?        00:00:00 ./tp1.out
root        1857    1856  0 23:21 ?        00:00:00 ./tp1.out
root        1858    1856  0 23:21 ?        00:00:00 ./tp1.out
root        1859    1856  0 23:21 ?        00:00:00 ./tp1.out
root        1860    1857  0 23:21 ?        00:00:00 ./tp1.out
root        1861    1857  0 23:21 ?        00:00:00 ./tp1.out
root        1862    1859  0 23:21 ?        00:00:00 ./tp1.out
root        1898    1712  0 23:21 ?        00:00:00 /bin/bash -c ps -ef | grep tp1.out
root        1900    1898  0 23:21 ?        00:00:00 grep tp1.out


In [ ]:
!pstree -pc $(pgrep -f tp1.out -P 1)

tp1.out(1856)─┬─tp1.out(1857)─┬─tp1.out(1860)
              │               └─tp1.out(1861)
              ├─tp1.out(1858)
              └─tp1.out(1859)───tp1.out(1862)


# **Python**

In [ ]:
%%writefile tp1.py
import os
import time

def processMessage(node):
    message = "Hola mi nombre es {name}. PID: {pid}. PPID {ppid}\n"
    print(message.format(name=node,pid=os.getpid(),ppid=os.getppid()))

def spawnProcesses(node):
    pid=0
    name,children = node
    processMessage(name)
    for item in children:
        pid=os.fork()
        if pid < 0:
            print("Error al crear nuevo proceso")
            os._exit(1)
        if not pid:
            spawnProcesses(item)
            time.sleep(200)
            os._exit(0)
    for item in children:
        os.wait()

def main():
    tree = ("A", [("B", [("E",[]), ("F",[])]), ("C",[]), ("D", [("G",[])])])
    spawnProcesses(tree)
    os._exit(0)

main()

Writing tp1.py


In [ ]:
!touch resultado_py && chmod 700 resultado_py

In [ ]:
!nohup python3 -u tp1.py 1>resultado_py 2>/dev/null &

In [ ]:
!cat resultado_py

Hola mi nombre es A. PID: 503. PPID 1

Hola mi nombre es C. PID: 505. PPID 503

Hola mi nombre es D. PID: 506. PPID 503

Hola mi nombre es G. PID: 507. PPID 506

Hola mi nombre es B. PID: 504. PPID 503

Hola mi nombre es F. PID: 509. PPID 504

Hola mi nombre es E. PID: 508. PPID 504



In [ ]:
!ps -ef | grep tp1.py

root         503       1  1 01:36 ?        00:00:00 python3 -u tp1.py
root         504     503  0 01:36 ?        00:00:00 python3 -u tp1.py
root         505     503  0 01:36 ?        00:00:00 python3 -u tp1.py
root         506     503  0 01:36 ?        00:00:00 python3 -u tp1.py
root         507     506  0 01:36 ?        00:00:00 python3 -u tp1.py
root         508     504  0 01:36 ?        00:00:00 python3 -u tp1.py
root         509     504  0 01:36 ?        00:00:00 python3 -u tp1.py
root         531     348  0 01:36 ?        00:00:00 /bin/bash -c ps -ef | grep tp1.py
root         533     531  0 01:36 ?        00:00:00 grep tp1.py


In [ ]:
!pstree -pc $(pgrep -f tp1.py -P 1)

python3(503)─┬─python3(504)─┬─python3(508)
             │              └─python3(509)
             ├─python3(505)
             └─python3(506)───python3(507)


Comentarios:


*   Al principio python no imprimia el output en el archivo resultado_py porque le faltaba la opcion -u en el comando de ejecucion





# **Java**

In [ ]:
%%writefile Main.java
import java.io.IOException;

public class Main 
{
    public static void main(String[] args) 
    {
        //Construimos Proceso B
        ProcessBuilder processBuilderB = new ProcessBuilder("java", "ChildProcess.java", "B");
        processBuilderB.redirectErrorStream(true);
        processBuilderB.inheritIO();
  
        //Construimos Proceso C
        ProcessBuilder processBuilderC = new ProcessBuilder("java", "ChildProcess.java", "C");
        processBuilderC.redirectErrorStream(true);
        processBuilderC.inheritIO();
  
        //Construimos Proceso D
        ProcessBuilder processBuilderD = new ProcessBuilder("java", "ChildProcess.java", "D");
        processBuilderD.redirectErrorStream(true);
        processBuilderD.inheritIO();
  
        try 
        {
            // Se muestra información del Proceso
            System.out.println(getInfoProcess());
    
            //Se solicita al SO el inicio del Proceso B, C y D
            Process processB = processBuilderB.start();
            Process processC = processBuilderC.start();
            Process processD = processBuilderD.start();
    
            //Se espera señal de finalización de lo Procesos B, C y D
            processB.waitFor();
            processC.waitFor();
            processD.waitFor();
            Thread.sleep(10000);
        } 
        //Tratamiento de excepciones
        catch (IOException ex) 
        {
            System.out.println("Error al iniciar Proceso Hijo. Mensaje de error: " + ex.getMessage());
        } 
        catch (InterruptedException ex) 
        {
            System.out.println("Error en realizar waitFor sobre el proceso hijo. Mensaje de error: " + ex.getMessage());
        }
    }

    //Método para obtener información del proceso (PID, PPID)
    public static String getInfoProcess() 
    {
        ProcessHandle processHandle = ProcessHandle.current();
        long pid = processHandle.pid();
        long ppid = processHandle.parent().get().pid();
        return String.format("Proceso A | PID: %s | PPID: %s", pid, ppid);
    }
}

Overwriting Main.java


In [ ]:
%%writefile ChildProcess.java
import java.io.IOException;

public class ChildProcess 
{
    //Definimos constantes a utilizar
    private static final String PROCESS_B = "B";
    private static final String PROCESS_D = "D";

    public static void main(String[] args) 
    {
        //Obtenemos el nombre del proceso recibido por parámetro
        String processName = args[0];
        //Mostramos la información del Proceso
        System.out.println("Proceso " + processName + " | " + getInfoProcess());
        switch (processName) 
        {
            //Lógica asociada a la ejecución del Proceso B
            case PROCESS_B:
                execProcessB();
                break;
            //Lógica asociada a la ejecución del Proceso D
            case PROCESS_D:
                execProcessD();
                break;
            default:
                break;
        }
        //Sleep para verificar resultado y tratamiento de excepción 
        try 
        {
            Thread.sleep(10000);
        } 
        catch (InterruptedException ex) 
        {
            System.out.println("Error en sleep. Mensaje: " + ex.getMessage());
        }
    }

    //Método para obtener información del proceso (PID y PPID)
    public static String getInfoProcess() 
    {
        ProcessHandle processHandle = ProcessHandle.current();
        long pid = processHandle.pid();
        long ppid = processHandle.parent().get().pid();
        return String.format("PID: %s | PPID: %s", pid, ppid);
    }

    //Método con las acciones a realizar por el Proceso B
    public static void execProcessB() 
    {
        //Se construye Proceso E
        ProcessBuilder processBuilderE = new ProcessBuilder("java", "ChildProcess.java", "E");
        processBuilderE.redirectErrorStream(true);
        processBuilderE.inheritIO();

        //Se construye Proceso F
        ProcessBuilder processBuilderF = new ProcessBuilder("java", "ChildProcess.java", "F");
        processBuilderF.redirectErrorStream(true);
        processBuilderF.inheritIO();

        try 
        {
            //Se solicita al SO la ejecución del Proceso E y Proceso F
            Process processE = processBuilderE.start();
            Process processF = processBuilderF.start();
  
            //Se espera señal de finalización de los procesos E y F
            processE.waitFor();
            processF.waitFor();
        } 
        //Tratamiento de excepciones
        catch (IOException ex) 
        {
            System.out.println("Error al iniciar Proceso Hijo. Mensaje de error: " + ex.getMessage());
        } 
        catch (InterruptedException ex) 
        {
            System.out.println("Error en realizar waitFor sobre el proceso hijo. Mensaje de error: " + ex.getMessage());
        }
    }

    //Método con las acciones a realizar por el Proceso D
    public static void execProcessD() 
    {
        //Se construye Proceso G
        ProcessBuilder processBuilderG = new ProcessBuilder("java", "ChildProcess.java", "G");
        processBuilderG.redirectErrorStream(true);
        processBuilderG.inheritIO();
  
        try 
        {
            //Se solicita al SO la ejecución del Proceso G
            Process processG = processBuilderG.start();
    
            //Se espera señal de finalización del Proceso G
            processG.waitFor();
        } 
        //Tratamiento de excepciones
        catch (IOException ex) 
        {
            System.out.println("Error al iniciar Proceso Hijo. Mensaje de error: " + ex.getMessage());
        } 
        catch (InterruptedException ex) 
        {
            System.out.println("Error en realizar waitFor sobre el proceso hijo. Mensaje de error: " + ex.getMessage());
        }
    }
}

Overwriting ChildProcess.java


In [ ]:
!nohup java Main.java 1>salidaJava 2>/dev/null &

In [ ]:
!cat salidaJava

Proceso A | PID: 4860 | PPID: 1
Proceso C | PID: 4885 | PPID: 4860
Proceso B | PID: 4882 | PPID: 4860
Proceso D | PID: 4888 | PPID: 4860
Proceso E | PID: 4954 | PPID: 4882
Proceso F | PID: 4957 | PPID: 4882
Proceso G | PID: 4989 | PPID: 4888


In [ ]:
!pstree -pT $(pgrep -f Main.java -P 1)

java(4860)─┬─java(4882)─┬─java(4954)
           │            └─java(4957)
           ├─java(4885)
           └─java(4888)───java(4989)
